In [1]:
import bcrypt

def hash_password(password):
    # Generate a random salt and hash the password
    salt = bcrypt.gensalt()
    hashed_password = bcrypt.hashpw(password.encode('utf-8'), salt)

    return hashed_password

def check_password(password, hashed_password):
    # Check if the provided password matches the hashed password
    return bcrypt.checkpw(password.encode('utf-8'), hashed_password)


In [5]:
# Example usage
password_to_hash = "my_secure_password"
hashed_password = hash_password(password_to_hash)

print("Password: " + password_to_hash)
print("Hashed password: " + hashed_password.decode('utf-8'))

Password: my_secure_password
Hashed password: $2b$12$8eT.XE4Vgs..4rUc3tzd2eLIlUQfQADsI9m0oK0grX4DeFxse42fK


In [2]:
if check_password("my_secure_password", "$2b$12$8eT.XE4Vgs..4rUc3tzd2eLIlUQfQADsI9m0oK0grX4DeFxse42fK".encode('utf-8')):
    print("Password is correct.")
else:
    print("Incorrect password.")


Password is correct.


In [1]:
from flask_jwt_extended import decode_token

: 

In [ ]:
decode_token()